# Imports

In [44]:
import numpy as np
import pandas as pd
from mod17 import MOD17
from datetime import datetime


# Santarem

In [45]:
observacoes_santarem = pd.read_csv('dados/Ameriflux/santarem/FLX_BR-Sa1_FLUXNET2015_FULLSET_DD_2002-2011_1-4.csv', na_values=-9999)
observacoes_santarem.index = observacoes_santarem['TIMESTAMP'].apply(lambda x:datetime.strptime(str(x), '%Y%m%d'))
observacoes_santarem = observacoes_santarem["GPP_DT_VUT_REF"].dropna()
observacoes_santarem = observacoes_santarem.rename_axis('index')
observacoes_santarem = observacoes_santarem.rename('GPP')
observacoes_santarem.to_csv('dados_calculados_torres/observacoes_santarem_diario.csv')

sant_8D_2002 = observacoes_santarem['2002'].resample('8D').sum()
sant_8D_2003 = observacoes_santarem['2003'].resample('8D').sum()
sant_8D_2004 = observacoes_santarem['2004'].resample('8D').sum()
sant_8D_2005 = observacoes_santarem['2005'].resample('8D').sum()
sant_8D_2006 = observacoes_santarem['2006'].resample('8D').sum()
sant_8D_2008 = observacoes_santarem['2008'].resample('8D').sum()
sant_8D_2009 = observacoes_santarem['2009'].resample('8D').sum()
sant_8D_2010 = observacoes_santarem['2010'].resample('8D').sum()
sant_8D_2011 = observacoes_santarem['2011'].resample('8D').sum()

observacoes_santarem = pd.concat([sant_8D_2002, sant_8D_2003,
                                  sant_8D_2004, sant_8D_2005,
                                  sant_8D_2006, sant_8D_2008,
                                  sant_8D_2009, sant_8D_2010, sant_8D_2011], axis=0)

observacoes_santarem.to_csv('dados_calculados_torres/observacoes_santarem.csv')


In [46]:
# Função para converter datas no formato MODIS
def modis_date(data_str):
    ano = int(data_str[1:5])
    dia_ano = int(data_str[5:])
    data = datetime.strptime('{}-{}'.format(ano, dia_ano), '%Y-%j')
    return data

Gpp_mod = pd.read_csv('dados/MODIS/GPP/Santarem_GPP.csv', header=None,na_values='F') # unidade: (%)
Gpp_mod.index = Gpp_mod[2].apply(modis_date)  # Convertendo a data no formato MODIS

# Arrumando o dataframe
# ================================
Gpp_mod = Gpp_mod.drop([0,1,2,3,4], axis=1)
Gpp_mod.columns = range(len(Gpp_mod.columns))
Gpp_mod = Gpp_mod[144].to_frame().rename(columns={144:'Gpp_modis'})
sant_gpp_mod = Gpp_mod['2002':'2011'] * 0.0001 * 1000
mod17_sant = sant_gpp_mod['Gpp_modis']
mod17_sant = pd.concat([mod17_sant['2002':'2006'] ,mod17_sant['2008':'2011']], axis=0)
mod17_sant = mod17_sant.rename_axis('index')
mod17_sant = mod17_sant.rename('GPP')
mod17_sant.to_csv('dados_calculados_torres/gpp_modis_santarem.csv')

In [47]:
def constructor_2(path):
    nome = path.split('/')[1].split('_')[0]
    df = pd.read_csv(path, parse_dates=['date'])
    df = df.rename({'albedo':nome},axis=1)
    # Definindo a coluna 'system:time_start' como índice
    hora_inicio = df['date'].min().floor('d')
    df.index = hora_inicio + pd.to_timedelta(df.index, unit='h')
    # Removendo a coluna 'system:time_start'
    df = df.drop(['date','.geo'], axis=1)
    df = df.drop('system:index',axis=1)
    return df

# Leitura e pré-processamento dos dados de temperatura
tday = constructor_2('dados/Merra/santarem/st_t10m.csv') - 273.15 #( K° -> C°)
tday1 = tday['2002':'2006']
tday2 = tday['2008':'2011']
tday = pd.concat([tday['2002':'2006'],tday['2008':'2011']], axis=0)
tmin1 = tday1['T10M'].resample('D').min().values  # Reamostrando para Temperatura mínima diária
tmin2 = tday2['T10M'].resample('D').min().values  # Reamostrando para Temperatura mínima diária
tmin = np.concatenate([tmin1,tmin2])
tmean_caxiuana = tday['T10M'].resample('D').mean().values  # Temperatura média diária
tmean_vpd1 = tday1['T10M'].between_time('10:00', '22:00').resample('D').mean().values
tmean_vpd2 = tday2['T10M'].between_time('10:00', '22:00').resample('D').mean().values
tmean_vpd = np.concatenate([tmean_vpd1,tmean_vpd2])

# Leitura e pré-processamento dos dados de razão de mistura de vapor de água
qv10m = constructor_2('dados/Merra/santarem/st_qv10m.csv') # unidade: "Mass fraction" (kg/kg)
qv10m1 = qv10m['2002':'2006']
qv10m2 = qv10m['2008':'2011']
qv10m1 = pd.to_numeric(qv10m1['QV10M']).between_time('10:00', '22:00').resample('D').mean().values
qv10m2 = pd.to_numeric(qv10m2['QV10M']).between_time('10:00', '22:00').resample('D').mean().values
qv10m = np.concatenate([qv10m1, qv10m2])

# Leitura e pré-processamento dos dados de pressão atmosférica
ps = constructor_2('dados/Merra/santarem/st_ps.csv') # unidade: Pa  # Removendo vírgulas dos valores
ps1 = ps['2002':'2006']
ps2 = ps['2008':'2011']
ps1['PS'] = pd.to_numeric(ps1['PS'])
ps2['PS'] = pd.to_numeric(ps2['PS'])
ps1 = ps1.between_time('10:00', '22:00').resample('D').mean()
ps2 = ps2.between_time('10:00', '22:00').resample('D').mean()
ps1 = pd.to_numeric(ps1['PS']).values
ps2 = pd.to_numeric(ps2['PS']).values
ps = np.concatenate([ps1,ps2])

# Leitura e pré-processamento dos dados de radiação solar incidente
SWGNT = constructor_2('dados/Merra/santarem/st_swgnt.csv')
SWGNT1 = SWGNT['2002':'2006']
SWGNT2 = SWGNT['2008':'2011']
SWGNT1 = SWGNT1['SWGNT'].resample('D').mean().values # unidade: W/m^2
SWGNT2 = SWGNT2['SWGNT'].resample('D').mean().values # unidade: W/m^2
SWGNT = np.concatenate([SWGNT1,SWGNT2])

# Função para converter datas no formato MODIS
def modis_date(data_str):
    ano = int(data_str[1:5])
    dia_ano = int(data_str[5:])
    data = datetime.strptime('{}-{}'.format(ano, dia_ano), '%Y-%j')
    return data

# Leitura e pré-processamento dos dados de Fração de Absorção de Luz Fotossinteticamente Ativa (FPAR)
fpar = pd.read_csv('dados/Merra/santarem/st_fpar.csv', header=None,na_values='F') # unidade: (%)
fpar.index = fpar[2].apply(modis_date)  # Convertendo a data no formato MODIS

# Arrumando o dataframe
# ================================
fpar = fpar.drop([0,1,2,3,4], axis=1)
fpar.columns = range(len(fpar.columns))
fpar = fpar[144]

# ================================

# Tratamento de dados faltando usando interpolação linear (8 dias -> diário)
fpar = pd.concat([fpar.resample('D').interpolate('linear')['2002':'2006'], fpar.resample('D').interpolate('linear')['2008':'2011']], axis=0)
fpar = fpar.values


drivers_santarem = [
    fpar[:][...,None],
    tmin[:][...,None],
    MOD17.vpd(qv10m, ps, tmean_vpd)[...,None],
    MOD17.par(SWGNT)[:][...,None]
]
drivers_santarem[2] = np.where(drivers_santarem[2] < 0, 0, drivers_santarem[2]) # VPD < 0 = 0

# Gerando GPP a partir dos dados do bioma "Evergreen Broadleaf Forest" como parâmetro diretamente
parametros = pd.read_csv('mod17_bplut/MOD17_BPLUT_CX.X_MERRA_NASA.csv')['EBF=1'].values

parametros = [0.001405, -8.0, 9.09, 1000.0, 4000.0, 26.9, 2.0, 2.0, 1.1, 0.162, 0.00604, 0.00519, 0.00397] # Igual ao acima

gpp = MOD17._gpp(parametros,drivers_santarem[0],drivers_santarem[1],drivers_santarem[2],drivers_santarem[3])

index_santarem = pd.date_range(start='2002-01-01', end='2006-12-31', freq='D')
index_santarem = index_santarem.union(pd.date_range(start='2008-01-01', end='2011-12-31', freq='D'))

gpp_santarem = pd.DataFrame(gpp, index=index_santarem)

gpp_santarem = gpp_santarem.rename_axis('index')
gpp_santarem = gpp_santarem[0].rename('GPP')

gpp_santarem.to_csv('dados_calculados_torres/gpp_algpad_santarem_diario.csv')

np.save('dados_calculados_torres/drivers_santarem', drivers_santarem)

sant_8D_2002 = gpp_santarem['2002'].resample('8D').sum()
sant_8D_2003 = gpp_santarem['2003'].resample('8D').sum()
sant_8D_2004 = gpp_santarem['2004'].resample('8D').sum()
sant_8D_2005 = gpp_santarem['2005'].resample('8D').sum()
sant_8D_2006 = gpp_santarem['2006'].resample('8D').sum()
sant_8D_2008 = gpp_santarem['2008'].resample('8D').sum()
sant_8D_2009 = gpp_santarem['2009'].resample('8D').sum()
sant_8D_2010 = gpp_santarem['2010'].resample('8D').sum()
sant_8D_2011 = gpp_santarem['2011'].resample('8D').sum()

gpp_santarem = pd.concat([sant_8D_2002, sant_8D_2003,
                            sant_8D_2004, sant_8D_2005,
                            sant_8D_2006, sant_8D_2008,
                            sant_8D_2009, sant_8D_2010, sant_8D_2011], axis=0)

gpp_santarem.to_csv('dados_calculados_torres/gpp_algpad_santarem.csv')


C:\Users\Alan Breno\AppData\Local\Temp\ipykernel_12128\3213685025.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ps1['PS'] = pd.to_numeric(ps1['PS'])
C:\Users\Alan Breno\AppData\Local\Temp\ipykernel_12128\3213685025.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ps2['PS'] = pd.to_numeric(ps2['PS'])


In [48]:

drivers_santarem = np.load('dados_calculados_torres/drivers_santarem.npy')
dict = {'1': np.squeeze(drivers_santarem[0]), '2': np.squeeze(drivers_santarem[1]), '3': np.squeeze(drivers_santarem[2]), '4': np.squeeze(drivers_santarem[3])} 
df_drivers_santarem = pd.DataFrame(dict, index=index_santarem)

drivers_santarem_mes = []
for i in range(12):
    mes = df_drivers_santarem[df_drivers_santarem.index.month == i+1].to_numpy().T
    drivers_santarem_mes.append(mes)

np.savez('dados_calculados_torres/drivers_santarem_mes',    drivers_santarem_mes[0], 
                                                            drivers_santarem_mes[1],
                                                            drivers_santarem_mes[2],
                                                            drivers_santarem_mes[3],
                                                            drivers_santarem_mes[4],
                                                            drivers_santarem_mes[5],
                                                            drivers_santarem_mes[6],
                                                            drivers_santarem_mes[7],
                                                            drivers_santarem_mes[8],
                                                            drivers_santarem_mes[9],
                                                            drivers_santarem_mes[10],
                                                            drivers_santarem_mes[11])



# Peru

In [49]:
observacoes_peru = pd.read_csv('dados/Ameriflux/amazonia_peruana/AMF_PE-QFR_FLUXNET_FULLSET_DD_2018-2019_3-5.csv')
observacoes_peru.index = observacoes_peru['TIMESTAMP'].apply(lambda x:datetime.strptime(str(x), '%Y%m%d'))
observacoes_peru = observacoes_peru['GPP_DT_VUT_REF'].dropna()
observacoes_peru = observacoes_peru.rename_axis('index')
observacoes_peru = observacoes_peru.rename('GPP')
observacoes_peru.to_csv('dados_calculados_torres/observacoes_peru_diario.csv')

peru_2018 = observacoes_peru['2018'].resample('8D').sum()
peru_2019 = observacoes_peru['2019'].resample('8D').sum()

observacoes_peru = pd.concat([peru_2018, peru_2019], axis=0)

observacoes_peru.to_csv('dados_calculados_torres/observacoes_peru.csv')


In [50]:
# Função para converter datas no formato MODIS
def modis_date(data_str):
    ano = int(data_str[1:5])
    dia_ano = int(data_str[5:])
    data = datetime.strptime('{}-{}'.format(ano, dia_ano), '%Y-%j')
    return data

Gpp_mod = pd.read_csv('dados/MODIS/GPP/Peru_GPP.csv', header=None,na_values='F') # unidade: (%)
Gpp_mod.index = Gpp_mod[2].apply(modis_date)  # Convertendo a data no formato MODIS

# Arrumando o dataframe
# ================================
Gpp_mod = Gpp_mod.drop([0,1,2,3,4], axis=1)
Gpp_mod.columns = range(len(Gpp_mod.columns))
Gpp_mod = Gpp_mod[144].to_frame().rename(columns={144:'Gpp_modis'})
peru_gpp_mod = Gpp_mod['2018':'2019'] * 0.0001 * 1000
mod17_peru = peru_gpp_mod['Gpp_modis']
mod17_peru = mod17_peru.rename_axis('index')
mod17_peru = mod17_peru.rename('GPP')

mod17_peru.to_csv('dados_calculados_torres/gpp_modis_peru.csv')

In [51]:
def constructor_2(path):
    nome = path.split('/')[1].split('_')[0]
    df = pd.read_csv(path, parse_dates=['date'])
    df = df.rename({'albedo':nome},axis=1)
    # Definindo a coluna 'system:time_start' como índice
    hora_inicio = df['date'].min().floor('d')
    df.index = hora_inicio + pd.to_timedelta(df.index, unit='h')
    # Removendo a coluna 'system:time_start'
    df = df.drop(['date','.geo'], axis=1)
    df = df.drop('system:index',axis=1)
    return df

# Leitura e pré-processamento dos dados de temperatura
tday = constructor_2('dados/Merra/amazonia peruana/pe_t10m.csv') - 273.15 #( K° -> C°)
tday = tday['2018':'2019']
tmin = tday['T10M'].resample('D').min().values  # Reamostrando para Temperatura mínima diária
tmean_peru = tday['T10M'].resample('D').mean().values  # Temperatura média diária
tmean_vpd = tday['T10M'].between_time('10:00', '22:00').resample('D').mean().values

# Leitura e pré-processamento dos dados de razão de mistura de vapor de água
qv10m = constructor_2('dados/Merra/amazonia peruana/pe_qv10m.csv') # unidade: "Mass fraction" (kg/kg)
qv10m = qv10m['2018':'2019']
qv10m = pd.to_numeric(qv10m['QV10M']).between_time('10:00', '22:00').resample('D').mean().values

# Leitura e pré-processamento dos dados de pressão atmosférica
ps = constructor_2('dados/Merra/amazonia peruana/pe_ps.csv') # unidade: Pa  # Removendo vírgulas dos valores
ps = ps['2018':'2019']
ps['PS'] = pd.to_numeric(ps['PS'])
ps = ps.between_time('10:00', '22:00').resample('D').mean()
ps = pd.to_numeric(ps['PS']).values

# Leitura e pré-processamento dos dados de radiação solar incidente
SWGNT = constructor_2('dados/Merra/amazonia peruana/pe_swgnt.csv')
SWGNT = SWGNT['2018':'2019']
SWGNT = SWGNT['SWGNT'].resample('D').mean().values # unidade: W/m^2

# Função para converter datas no formato MODIS
def modis_date(data_str):
    ano = int(data_str[1:5])
    dia_ano = int(data_str[5:])
    data = datetime.strptime('{}-{}'.format(ano, dia_ano), '%Y-%j')
    return data

# Leitura e pré-processamento dos dados de Fração de Absorção de Luz Fotossinteticamente Ativa (FPAR)
fpar = pd.read_csv('dados/Merra/amazonia peruana/pe_fpar.csv', header=None,na_values='F') # unidade: (%)
fpar.index = fpar[2].apply(modis_date)  # Convertendo a data no formato MODIS

# Arrumando o dataframe
# ================================
fpar = fpar.drop([0,1,2,3,4], axis=1)
fpar.columns = range(len(fpar.columns))
fpar = fpar[144]

# ================================

# Tratamento de dados faltando usando interpolação linear (8 dias -> diário)
fpar = fpar.resample('D').interpolate('linear')['2018':'2019']

fpar = fpar.values

drivers_peru = [
    fpar[:][...,None],
    tmin[:][...,None],
    MOD17.vpd(qv10m, ps, tmean_vpd)[...,None],
    MOD17.par(SWGNT)[:][...,None]
]
drivers_peru[2] = np.where(drivers_peru[2] < 0, 0, drivers_peru[2]) # VPD < 0 = 0

# Gerando GPP a partir dos dados do bioma "Evergreen Broadleaf Forest" como parâmetro diretamente
parametros = pd.read_csv('mod17_bplut/MOD17_BPLUT_CX.X_MERRA_NASA.csv')['EBF=1'].values

parametros = [0.001405, -8.0, 9.09, 1000.0, 4000.0, 26.9, 2.0, 2.0, 1.1, 0.162, 0.00604, 0.00519, 0.00397] # Igual ao acima

gpp_ebf = MOD17._gpp(parametros,drivers_peru[0],drivers_peru[1],drivers_peru[2],drivers_peru[3]) # Considerar como GPP do algorítmo
index_peru = pd.date_range(start='2018-01-01', end='2019-12-31', freq='D')
gpp_peru = pd.DataFrame(gpp_ebf,index= index_peru)
gpp_peru = gpp_peru.rename_axis('index')
gpp_peru = gpp_peru[0].rename('GPP')
gpp_peru.to_csv('dados_calculados_torres/gpp_algpad_peru_diario.csv')

np.save('dados_calculados_torres/drivers_peru', drivers_peru)

peru_2018 = gpp_peru['2018'].resample('8D').sum()
peru_2019 = gpp_peru['2019'].resample('8D').sum()

gpp_peru = pd.concat([peru_2018, peru_2019], axis=0)

gpp_peru.to_csv('dados_calculados_torres/gpp_algpad_peru.csv')


In [52]:

drivers_peru = np.load('dados_calculados_torres/drivers_peru.npy')
dict = {'1': np.squeeze(drivers_peru[0]), '2': np.squeeze(drivers_peru[1]), '3': np.squeeze(drivers_peru[2]), '4': np.squeeze(drivers_peru[3])} 
df_drivers_peru = pd.DataFrame(dict, index=index_peru)

drivers_peru_mes = []
for i in range(12):
    mes = df_drivers_peru[df_drivers_peru.index.month == i+1].to_numpy().T
    drivers_peru_mes.append(mes)

np.savez('dados_calculados_torres/drivers_peru_mes',    drivers_peru_mes[0], 
                                                        drivers_peru_mes[1],
                                                        drivers_peru_mes[2],
                                                        drivers_peru_mes[3],
                                                        drivers_peru_mes[4],
                                                        drivers_peru_mes[5],
                                                        drivers_peru_mes[6],
                                                        drivers_peru_mes[7],
                                                        drivers_peru_mes[8],
                                                        drivers_peru_mes[9],
                                                        drivers_peru_mes[10],
                                                        drivers_peru_mes[11])



# Caxiuana

In [53]:
cax = pd.read_csv('dados/Ameriflux/caxiuana/GEPCAX.txt', delimiter='\t')
cax['ano2'] = cax['ano2'].apply(lambda x:str(int(x)))
cax['dia2'] = cax['dia2'].apply(lambda x:str(int(x)))
cax['datetime'] = cax['ano2'] + cax['dia2']
cax.index =  cax['datetime'].apply(lambda x:datetime.strptime(str(x), '%Y%j'))

observacoes_cax = cax['GEP']
observacoes_cax = observacoes_cax.rename_axis('index')
observacoes_cax = observacoes_cax.rename('GPP')

observacoes_cax.to_csv('dados_calculados_torres/observacoes_caxiuana_diario.csv')

cax_2005 = observacoes_cax['2005'].resample('8D').sum()
cax_2006 = observacoes_cax['2006'].resample('8D').sum()
cax_2007 = observacoes_cax['2007'].resample('8D').sum()
cax_2008 = observacoes_cax['2008'].resample('8D').sum()

observacoes_cax = pd.concat([cax_2005, cax_2006, cax_2007, cax_2008], axis=0)

observacoes_cax.to_csv('dados_calculados_torres/observacoes_caxiuana.csv')


In [54]:
# Função para converter datas no formato MODIS
def modis_date(data_str):
    ano = int(data_str[1:5])
    dia_ano = int(data_str[5:])
    data = datetime.strptime('{}-{}'.format(ano, dia_ano), '%Y-%j')
    return data

Gpp_mod = pd.read_csv('dados/MODIS/GPP/Caxiuana_GPP.csv', header=None, na_values='F') # unidade: (%)
Gpp_mod.index = Gpp_mod[2].apply(modis_date)  # Convertendo a data no formato MODIS

# Arrumando o dataframe
# ================================
Gpp_mod = Gpp_mod.drop([0,1,2,3,4], axis=1)
Gpp_mod.columns = range(len(Gpp_mod.columns))
Gpp_mod = Gpp_mod[144].to_frame().rename(columns={144:'Gpp_modis'})
caxiuana_gpp_mod = Gpp_mod['2005':'2008'] * 0.0001 * 1000
mod17_caxiuana = caxiuana_gpp_mod['Gpp_modis']
mod17_caxiuana = mod17_caxiuana.rename_axis('index')
mod17_caxiuana = mod17_caxiuana.rename('GPP')
mod17_caxiuana.to_csv('dados_calculados_torres/gpp_modis_caxiuana.csv')

In [55]:
def constructor_2(path):
    nome = path.split('/')[1].split('_')[0]
    df = pd.read_csv(path, parse_dates=['date'])
    df = df.rename({'albedo':nome},axis=1)
    # Definindo a coluna 'system:time_start' como índice
    hora_inicio = df['date'].min().floor('d')
    df.index = hora_inicio + pd.to_timedelta(df.index, unit='h')
    # Removendo a coluna 'system:time_start'
    df = df.drop(['date','.geo'], axis=1)
    df = df.drop('system:index',axis=1)
    return df

# Leitura e pré-processamento dos dados de temperatura
tday = constructor_2('dados/Merra/caxiuana/cax_t10m.csv') - 273.15 #( K° -> C°)
tday = tday['2005':'2008']
tmin = tday['T10M'].resample('D').min().values  # Reamostrando para Temperatura mínima diária
tmean_caxiuana = tday['T10M'].resample('D').mean().values  # Temperatura média diária
tmean_vpd = tday['T10M'].between_time('10:00', '22:00').resample('D').mean().values

# Leitura e pré-processamento dos dados de razão de mistura de vapor de água
qv10m = constructor_2('dados/Merra/caxiuana/cax_qv10m.csv') # unidade: "Mass fraction" (kg/kg)
qv10m = qv10m['2005':'2008']
qv10m = pd.to_numeric(qv10m['QV10M']).between_time('10:00', '22:00').resample('D').mean().values

# Leitura e pré-processamento dos dados de pressão atmosférica
ps = constructor_2('dados/Merra/caxiuana/cax_ps.csv') # unidade: Pa  # Removendo vírgulas dos valores
ps = ps['2005':'2008']
ps['PS'] = pd.to_numeric(ps['PS'])
ps = ps.between_time('10:00', '22:00').resample('D').mean()
ps = pd.to_numeric(ps['PS']).values

# Leitura e pré-processamento dos dados de radiação solar incidente
SWGNT = constructor_2('dados/Merra/caxiuana/cax_swgnt.csv')
SWGNT = SWGNT['2005':'2008']
SWGNT = SWGNT['SWGNT'].resample('D').mean().values # unidade: W/m^2

# Função para converter datas no formato MODIS
def modis_date(data_str):
    ano = int(data_str[1:5])
    dia_ano = int(data_str[5:])
    data = datetime.strptime('{}-{}'.format(ano, dia_ano), '%Y-%j')
    return data

# Leitura e pré-processamento dos dados de Fração de Absorção de Luz Fotossinteticamente Ativa (FPAR)
fpar = pd.read_csv('dados/Merra/caxiuana/cax_fpar.csv', header=None,na_values='F') # unidade: (%)
fpar.index = fpar[2].apply(modis_date)  # Convertendo a data no formato MODIS

# Arrumando o dataframe
# ================================
fpar = fpar.drop([0,1,2,3,4], axis=1)
fpar.columns = range(len(fpar.columns))
fpar = fpar[144]

# ================================

# Tratamento de dados faltando usando interpolação linear (8 dias -> diário)
fpar = fpar.resample('D').interpolate('linear')['2005':'2008']

fpar = fpar.values

drivers_caxiuana = [
    fpar[:][...,None],
    tmin[:][...,None],
    MOD17.vpd(qv10m, ps, tmean_vpd)[...,None],
    MOD17.par(SWGNT)[:][...,None]
]
drivers_caxiuana[2] = np.where(drivers_caxiuana[2] < 0, 0, drivers_caxiuana[2]) # VPD < 0 = 0

# Gerando GPP a partir dos dados do bioma "Evergreen Broadleaf Forest" como parâmetro diretamente
parametros = pd.read_csv('mod17_bplut/MOD17_BPLUT_CX.X_MERRA_NASA.csv')['EBF=1'].values

parametros = [0.001405, -8.0, 9.09, 1000.0, 4000.0, 26.9, 2.0, 2.0, 1.1, 0.162, 0.00604, 0.00519, 0.00397] # Igual ao acima

gpp_ebf = MOD17._gpp(parametros,drivers_caxiuana[0],drivers_caxiuana[1],drivers_caxiuana[2],drivers_caxiuana[3]) # Considerar como GPP do algorítmo
index_caxiuana = pd.date_range(start='2005-01-01', end='2008-12-31', freq='D')

gpp_caxiuana = pd.DataFrame(gpp_ebf,index=index_caxiuana)

gpp_caxiuana = gpp_caxiuana.rename_axis('index')
gpp_caxiuana = gpp_caxiuana[0].rename('GPP')

gpp_caxiuana.to_csv('dados_calculados_torres/gpp_algpad_caxiuana_diario.csv')

np.save('dados_calculados_torres/drivers_caxiuana', drivers_caxiuana)

cax_2005 = gpp_caxiuana['2005'].resample('8D').sum()
cax_2006 = gpp_caxiuana['2006'].resample('8D').sum()
cax_2007 = gpp_caxiuana['2007'].resample('8D').sum()
cax_2008 = gpp_caxiuana['2008'].resample('8D').sum()

gpp_caxiuana = pd.concat([cax_2005, cax_2006, cax_2007, cax_2008], axis=0)

gpp_caxiuana.to_csv('dados_calculados_torres/gpp_algpad_caxiuana.csv')


In [56]:
drivers_caxiuana = np.load('dados_calculados_torres/drivers_caxiuana.npy')
dict = {'1': np.squeeze(drivers_caxiuana[0]), '2': np.squeeze(drivers_caxiuana[1]), '3': np.squeeze(drivers_caxiuana[2]), '4': np.squeeze(drivers_caxiuana[3])} 
df_drivers_caxiuana = pd.DataFrame(dict, index=index_caxiuana)

drivers_caxiuana_mes = []
for i in range(12):
    mes = df_drivers_caxiuana[df_drivers_caxiuana.index.month == i+1].to_numpy().T
    drivers_caxiuana_mes.append(mes)

np.savez('dados_calculados_torres/drivers_caxiuana_mes',    drivers_caxiuana_mes[0], 
                                                            drivers_caxiuana_mes[1],
                                                            drivers_caxiuana_mes[2],
                                                            drivers_caxiuana_mes[3],
                                                            drivers_caxiuana_mes[4],
                                                            drivers_caxiuana_mes[5],
                                                            drivers_caxiuana_mes[6],
                                                            drivers_caxiuana_mes[7],
                                                            drivers_caxiuana_mes[8],
                                                            drivers_caxiuana_mes[9],
                                                            drivers_caxiuana_mes[10],
                                                            drivers_caxiuana_mes[11])

